In [1]:
import os


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START

In [4]:
def create_agent(llm, tools, system_message: str):
    """Create an Agent"""
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """You are a helpful AI assistant, collabrating with other assistants.
            Use the provided tools to progress towards answering the question.
            If you are unable to fully answer, that's OK, another assistant with different tools will help where you left off.
            Execute what you can to make progress.
            If you or any of the other assistants have the final answer or deliverable,
            prefix your response with FINAL ANSWER so the team knows to stop.
            You have access to the following tools: {tool_names}.\n{system_message},
            
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ])
    prompt=prompt.partial(system_message=system_message)
    prompt=prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)


# Define Tools

In [5]:
from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_resutls=5)

In [12]:
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

repl = PythonREPL()

@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    """
    Execute the given Python code in a REPL environment and return the result.
    
    Args:
        code (str): The Python code to execute.
    
    Returns:
        str: The result of the code execution or an error message.
    """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n````\nStdout: {result}"
    return (result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER.")
    

# Create graph
도구들을 정의하고 보조 함수를 만들었으므로, 아래에서 개별 에이전트들을 생성하고 LangGraph를 사용하여 서로 소통하는 방법

## Define State
먼저 그래프의 상태를 정의함, 이는 단순히 메시지 목록과 가장 최근에 메시지를 보낸 발신자(sender)를 추적하는 키로 구성됨

In [13]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain.chat_models import ChatOllama

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str #지금 메시지가 어떤 AI Agent에서 넘어왔는지 확인함

# Define Agent Nodes

In [14]:
import functools
from langchain_core.messages import AIMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)

    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name
    }

llm = ChatOllama(
    model="llama3.1:latest"
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2452\4054613889.py:18: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [15]:
research_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You should provide accurate data for use, "
        "and source code shouldn't be the final answer",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

NotImplementedError: 